#  Calculating the Ne Spectral Lines Ratios and join with the results from CIGALE

-------------------

Import of the libraries

In [1]:
import numpy as np
import astropy.units as u

from astropy.table import Table, Column, table, vstack, join
from pathlib import Path
from matplotlib import pyplot as plt

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.container { font-size:18px !important; }</style>"))

Reading the different paths where the results has been saved

In [2]:
path_raw = '../Datos/Raw/'
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/'
path_SED = '../Datos/Work/SEDs/'
path_CIGALE_phot = '../Datos/Work/CIGALE_InputPhot/'
path_CIGALE_out = '../Datos/Work/CIGALE_Output/'
path_graphs = '../Datos/Final/Graphics/'
path_FINAL = '../Datos/Final/'

Reading the table with the galaxies where Ne lines info was founded

In [3]:
date_sample = 'Sep_02_2022'
Ne_Sy = Table.read(path_raw+'Obj_Sy_Ne-IR_'+date_sample+'.vot',format='votable')
print('Number of Seyfert Galaxies with Ne lines info:',len(Ne_Sy))

Number of Seyfert Galaxies with Ne lines info: 303


We remove the galaxies without energy balance that was removed in the CIGALE's phot files (Notebook 04).

In [4]:
CIG_BadGalaxies = Table.read('../Datos/Complementary/CIGPhot_BadEnergyBalance.tbl', format='ascii')
for id in CIG_BadGalaxies['id']:
    Ne_Sy.remove_rows(np.where(Ne_Sy['Main_Id'] == id)[0])
print('Number of Seyfert Galaxies with Ne lines info after remove bad galaxies:',len(Ne_Sy))

Number of Seyfert Galaxies with Ne lines info after remove bad galaxies: 252


Now we classify the tables according to their Ne line information

In [5]:
NeV_NeII_Sy = vstack([Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeII|NeIII|NeV'],\
                    Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeII|NeV']])

NeV_NeIII_Sy = vstack([Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeII|NeIII|NeV'],\
                    Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeIII|NeV']])

NeIII_NeII_Sy = vstack([Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeII|NeIII|NeV'],\
                    Ne_Sy[Ne_Sy['NED_Ne_IR_info']=='NeII|NeIII']])

print('Total objects with [NeV] and [NeII] info:',len(NeV_NeII_Sy))
print('Total objects with [NeV] and [NeIII] info:',len(NeV_NeIII_Sy))
print('Total objects with [NeIII] and [NeII] info:',len(NeIII_NeII_Sy))

Total objects with [NeV] and [NeII] info: 162
Total objects with [NeV] and [NeIII] info: 156
Total objects with [NeIII] and [NeII] info: 191


------------------------------
## Function for extracting the Neon lines flux values from NED phot tables

In [6]:
def Neon_info(NED_table, Freq):
    
    '''
    This function create a new table from the Phot NED table
    but with the flux of [NeII] and [NeV]
    '''
    
    NED_table.remove_rows(np.where(NED_table['Flux_Density'].mask)[0])
    NED_table.remove_rows(np.where(NED_table['NED_Units'] == 'Jy')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] line (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] 14.4 (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] (IRS)')[0])
    NED_table.remove_rows(np.where(NED_table['Observed_Passband'] == '[Cl II] 14.37 (IRS)')[0])
    
    Ne_idx = [np.where(NED_table['Frequency'] == freq)[0] for freq in Freq]
    #print(Ne_idx)
    #print(len(Ne_idx))
    i = []
    Ne_table = Table()
    for idx in range(len(Freq)):
        Ne_table = vstack([Ne_table,NED_table[Ne_idx[idx]]])
    
    for ind in range(len(Ne_table)):
        i.append(ind)
    
    Ne_table['idx'] = i
    Ne_inf = Ne_table['idx','Observed_Passband','Flux_Density','NED_Uncertainty','NED_Units','Refcode']
    obs_passb = np.unique(Ne_inf['Observed_Passband'])
    #print(obs_passb)
    return Ne_inf

--------------------------
## Function for calculating the Neon spectral line ratios:

In [7]:
def Ne_ratios(Ne_info,Nel1,Nel2,Nel1a,Nel2a,not_same_bib):
    
    '''
    This function is for compute the ratio between two Neon
    emission lines (line1/line2) that have the same bibcode
    '''
    #Extract the number value of the NED_Uncertainty
    for u in range(len(Ne_info)):
        #print(u,Ne_info['NED_Uncertainty'][u])
        if Ne_info['NED_Uncertainty'][u] != '':
            Ne_info['NED_Uncertainty'][u] = float(Ne_info['NED_Uncertainty'][u].split('+/-')[-1])
    
    bibcod, count = np.unique(Ne_info['Refcode'], return_counts=True) #Extract the bibcodes
    print('No. of [Ne] data',len(Ne_info))
    flag = 0 #Flag to detect when there is not info of the two Ne lines for the same bibcode
    
    #Print the lines passband names
    #print('Neon line 1:',Nel1,'& Neon line 2:',Nel2)
    #print('Others filter names for Neon line1:',Nel1a,'& Neon line 2:',Nel2a)
    
    line1val = []
    line2val = []
    Ne_ratio = []
    Ne_ratio_err = []
    Ne_filter = []
    refcode = []
    #print(Ne_info) #Print the Ne Info of each galaxy
    
    for idx,bib in enumerate(bibcod):
        i = np.where(Ne_info['Refcode'] == bib)[0]
        #The ratio [Ne]line1/[Ne]line2 is calculated for only the bibcodes that have both flux data
        if len(i)>=2:
            flag += 1
            #print(len(i))
            Neline1_i = 'nan'
            Neline2_i = 'nan'
            for j,ind in enumerate(i):
                if Nel1 in Ne_info['Observed_Passband'][ind] or Nel1a in Ne_info['Observed_Passband'][ind]:
                    #print('yes:',Nel1)
                    Neline1_i = ind
                elif Nel2 in Ne_info['Observed_Passband'][ind] or Nel2a in Ne_info['Observed_Passband'][ind]:
                    #print('yes:',Nel2)
                    Neline2_i = ind
            if Neline1_i != 'nan' and Neline2_i != 'nan':
                Neline1 = Ne_info['Flux_Density'][Neline1_i]
                Neline1_err = Ne_info['NED_Uncertainty'][Neline1_i]
                Neline2 = Ne_info['Flux_Density'][Neline2_i]
                Neline2_err = Ne_info['NED_Uncertainty'][Neline2_i]
                Neline1_Neline2 = Neline1/Neline2
               
                Ne_ratio.append(Neline1_Neline2)
                line1val.append(Neline1) #Value of the flux in line 1
                line2val.append(Neline2) #Value of the flux in line 2
                Ne_filter.append(Ne_info['Observed_Passband'][Neline1_i])
                Ne_filter.append(Ne_info['Observed_Passband'][Neline2_i])
                refcode.append(bib)
                #print('For bibcode',',',bib,Nel1,'index is:',Neline1_i,'and',Nel2,'index is:',Neline2_i)
                #print('For bibcode', bib,Nel1,'/',Nel2,'ratio is:',Neline1_Neline2)
                
                if Neline1_err != '' or Neline2_err != '':
                    Neline1_Neline2_err = np.absolute(Neline1_Neline2)*np.sqrt((Neline1_err/Neline1)**2+(Neline2_err/Neline2)**2)
                    #Neline1_Neline2_err = Neline1_err/Neline2_err
                else:
                    Neline1_Neline2_err = np.nan #Se usa np.nan para poder hacer operaciones mat con este
                Ne_ratio_err.append(Neline1_Neline2_err)
            
            #In some cases there is two data for one bibcode but for the same spectral line
            elif (Neline1_i == 'nan' and Neline2_i != 'nan') and len(Ne_info)==2:
                flag = 0
    if flag<1:
        not_same_bib += 1
        #for r in range(len(NeV_NeII_inf)): refcode.append(NeV_NeII_inf['Refcode'][r])
         
    return Ne_ratio, Ne_ratio_err, line1val, line2val, Ne_filter, refcode, not_same_bib

In the galaxies where there are more than one data of [NeII] or [NeV], we remove the rows if per bibcode there is data for only one the them. The idea is to have both [NeII] and [NeV] data per bibcode:

## [NeV]/[NeII]

In [8]:
Nefilter = []
Refcode = []
NeV_NeII = Table(names=('Main_id','RA','DEC','otype','redshift','[NeV]/[NeII]','[NeV]/[NeII]_err'),
                dtype=('O','f8','f8','O','f8','f8','f8'))
NeV_NeII_bib = Table(names=('Main_id','RA','DEC','otype','redshift','[NeV]/[NeII]','[NeV]/[NeII]_err','Bibcode'),
                dtype=('O','f8','f8','O','f8','f8','f8','O'))

not_same_bib = 0 #After call the function, it will save the number of galaxies with not same bibcode data

for id in range(len(NeV_NeII_Sy)):
    NED_phot = Table()
    obj = NeV_NeII_Sy['Main_Id'][id]
    ra = NeV_NeII_Sy['RA'][id]
    dec = NeV_NeII_Sy['DEC'][id]
    otype = NeV_NeII_Sy['otype_txt'][id]
    z = NeV_NeII_Sy['redshift'][id]
    print(id)
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    print('Phot table lenght:',len(NED_phot))
    
    NeV_NeII_Freq = [20900000000000.0,21000000000000.0,23400000000000.0]
    NeV_NeII_inf = Neon_info(NED_phot, NeV_NeII_Freq)
    
    NeV_NeII_ratios,NeV_NeII_ratios_err,NeV,NeII,Ne_filter,refcode,not_same_bib = \
    Ne_ratios(NeV_NeII_inf,'[Ne V]','[Ne II]','nan','nan',not_same_bib)
    
    print('[NeV] flux density (Jy): ',NeV)
    print('[NeII] flux density (Jy): ',NeII)
    print('[NeV]/[NeII]: ',NeV_NeII_ratios)
    print('[NeV]/[NeII] errors: ',NeV_NeII_ratios_err)
    print('Bibcodes: ',refcode)
    
    for r in range(len(refcode)): Refcode.append(refcode[r])
    for f in range(len(Ne_filter)): Nefilter.append(Ne_filter[f])
        
    avg_Ne_ratios = np.average(NeV_NeII_ratios)
    std_Ne_ratios = np.std(NeV_NeII_ratios)
    if len(NeV_NeII_ratios)>1:
        NeV_NeII_ratios_err = np.nan_to_num(NeV_NeII_ratios_err)
    err = np.sqrt(np.sum(np.array(NeV_NeII_ratios_err)**2))/len(NeV_NeII_ratios_err)
    if err == 0: err = np.nan
    NeV_NeII_row = [obj,ra,dec,otype,z,avg_Ne_ratios,err]
    NeV_NeII.add_row(vals=NeV_NeII_row)
    
    for b,ratio in enumerate(NeV_NeII_ratios):
        NeV_NeII_bib_row = [obj,ra,dec,otype,z,ratio,NeV_NeII_ratios_err[b],refcode[b]]
        NeV_NeII_bib.add_row(vals=NeV_NeII_bib_row)
        
    print('Average [NeV]/[NeII]: ',avg_Ne_ratios)
    print('Error: ',err)
    print('\n')
    
print('Number of galaxies with diferent bibcode for [NeV] and [NeII] fluxes:',not_same_bib)
print('Galaxies idx with diferent bibcode for [NeV] and [NeII] fluxes:',\
      np.where(np.isnan(NeV_NeII['[NeV]/[NeII]']))[0])

#Uncomment the 'for' lines for print the 'Observed_passband'and the Ŕefcodes' with [NeV] and [NeII] information
Ne_filters1, countspb1 = np.unique(Nefilter, return_counts=True)
#for pb in range(len(Ne_filters1)): print(Ne_filters1[pb])
    
refcodes1, countsrc1 = np.unique(Refcode, return_counts=True)
#for rc in range(len(refcodes1)): print(refcodes1[rc])

0
Galaxy:  3C 234.0
Phot table lenght: 253
No. of [Ne] data 4
[NeV] flux density (Jy):  [3300000000.0]
[NeII] flux density (Jy):  [800000000.0]
[NeV]/[NeII]:  [4.125]
[NeV]/[NeII] errors:  [1.3524428130239001]
Bibcodes:  ['2006ApJ...647..161O']
Average [NeV]/[NeII]:  4.125
Error:  1.3524428130239001


1
Galaxy:  NGC   262
Phot table lenght: 334
No. of [Ne] data 7
[NeV] flux density (Jy):  [5820000000.0, 6900000000.0, 5820000000.0]
[NeII] flux density (Jy):  [16400000000.0, 15300000000.0, 16400000000.0]
[NeV]/[NeII]:  [0.3548780487804878, 0.45098039215686275, 0.3548780487804878]
[NeV]/[NeII] errors:  [0.022645479318087677, 0.03208429446652159, nan]
Bibcodes:  ['2010ApJ...709.1257T', '2010ApJ...716.1151W', '2011ApJS..195...17W']
Average [NeV]/[NeII]:  0.3869121632392794
Error:  0.013090368320768453


2
Galaxy:  IRAS 04385-0828
Phot table lenght: 97
No. of [Ne] data 5
[NeV] flux density (Jy):  [2280000000.0, 2280000000.0]
[NeII] flux density (Jy):  [13900000000.0, 13900000000.0]
[NeV]/[Ne

Phot table lenght: 558
No. of [Ne] data 13
[NeV] flux density (Jy):  [108000000000.0, 77100000000.0, 77000000000.0, 56000000000.0, 55000000000.0]
[NeII] flux density (Jy):  [135000000000.0, 134000000000.0, 131000000000.0, 118000000000.0, 118000000000.0]
[NeV]/[NeII]:  [0.8, 0.5753731343283582, 0.5877862595419847, 0.4745762711864407, 0.4661016949152542]
[NeV]/[NeII] errors:  [0.047868131618973374, nan, nan, 0.010497262325997694, nan]
Bibcodes:  ['2007ApJ...671..124D', '2009ApJ...690.1105K', '2010ApJ...725.2270P', '2011ApJ...740...94D', '2011ApJS..195...17W']
Average [NeV]/[NeII]:  0.5807674719944076
Error:  0.009801123488727543


26
Galaxy:  ESO 434-40
Phot table lenght: 211
No. of [Ne] data 2
[NeV] flux density (Jy):  [11100000000.0]
[NeII] flux density (Jy):  [18100000000.0]
[NeV]/[NeII]:  [0.6132596685082873]
[NeV]/[NeII] errors:  [0.03107418380785893]
Bibcodes:  ['2010ApJ...716.1151W']
Average [NeV]/[NeII]:  0.6132596685082873
Error:  0.03107418380785893


27
Galaxy:  ESO 323-77
Pho

/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_257178/2360053561.py:42: RuntimeWarning: invalid value encountered in double_scalars


No. of [Ne] data 2
[NeV] flux density (Jy):  [1420000000.0]
[NeII] flux density (Jy):  [4150000000.0]
[NeV]/[NeII]:  [0.3421686746987952]
[NeV]/[NeII] errors:  [0.037521080505244685]
Bibcodes:  ['2012ApJ...747...95G']
Average [NeV]/[NeII]:  0.3421686746987952
Error:  0.037521080505244685


33
Galaxy:  NGC   513
Phot table lenght: 124
No. of [Ne] data 5
[NeV] flux density (Jy):  [1520000000.0, 1910000000.0]
[NeII] flux density (Jy):  [10400000000.0, 12800000000.0]
[NeV]/[NeII]:  [0.14615384615384616, 0.14921875]
[NeV]/[NeII] errors:  [0.033389071572696924, nan]
Bibcodes:  ['2010ApJ...716.1151W', '2011ApJS..195...17W']
Average [NeV]/[NeII]:  0.14768629807692307
Error:  0.016694535786348462


34
Galaxy:  ESO 383-35
Phot table lenght: 271
No. of [Ne] data 11
[NeV] flux density (Jy):  [5080000000.0, 5010000000.0, 6050000000.0, 5010000000.0]
[NeII] flux density (Jy):  [3670000000.0, 4980000000.0, 4200000000.0, 4980000000.0]
[NeV]/[NeII]:  [1.3841961852861036, 1.0060240963855422, 1.4404761904

No. of [Ne] data 6
[NeV] flux density (Jy):  [1100000000.0, 1190000000.0]
[NeII] flux density (Jy):  [15000000000.0, 21100000000.0]
[NeV]/[NeII]:  [0.07333333333333333, 0.05639810426540284]
[NeV]/[NeII] errors:  [nan, nan]
Bibcodes:  ['2010ApJ...725.2270P', '2011ApJS..195...17W']
Average [NeV]/[NeII]:  0.06486571879936809
Error:  nan


56
Galaxy:  Mrk  304
Phot table lenght: 140
No. of [Ne] data 3
[NeV] flux density (Jy):  [270000000.0]
[NeII] flux density (Jy):  [226000000.0]
[NeV]/[NeII]:  [1.1946902654867257]
[NeV]/[NeII] errors:  [nan]
Bibcodes:  ['2006ApJ...649...79S']
Average [NeV]/[NeII]:  1.1946902654867257
Error:  nan


57
Galaxy:  Mrk 1095
Phot table lenght: 202
No. of [Ne] data 3
[NeV] flux density (Jy):  [1000000000.0]
[NeII] flux density (Jy):  [2900000000.0]
[NeV]/[NeII]:  [0.3448275862068966]
[NeV]/[NeII] errors:  [0.06314400266854273]
Bibcodes:  ['2011ApJ...740...94D']
Average [NeV]/[NeII]:  0.3448275862068966
Error:  0.06314400266854273


58
Galaxy:  IC  486
Phot table

...

/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_257178/2360053561.py:42: RuntimeWarning: invalid value encountered in double_scalars


No. of [Ne] data 16
[NeV] flux density (Jy):  [18000000000.0, 11300000000.0, 10700000000.0, 10200000000.0, 11000000000.0, 10800000000.0, 10700000000.0]
[NeII] flux density (Jy):  [17000000000.0, 16800000000.0, 21200000000.0, 19700000000.0, 17000000000.0, 17300000000.0, 21200000000.0]
[NeV]/[NeII]:  [1.0588235294117647, 0.6726190476190477, 0.5047169811320755, 0.5177664974619289, 0.6470588235294118, 0.6242774566473989, 0.5047169811320755]
[NeV]/[NeII] errors:  [0.171341431185328, 0.03200680723936727, 0.018083975533853824, 0.04090642225496949, nan, 0.028650487974464286, nan]
Bibcodes:  ['2007ApJ...671..124D', '2009MNRAS.398.1165G', '2010ApJ...709.1257T', '2010ApJ...716.1151W', '2010ApJ...725.2270P', '2011ApJ...740...94D', '2011ApJS..195...17W']
Average [NeV]/[NeII]:  0.6471399024191005
Error:  0.026031197046605253


121
Galaxy:  LEDA   70560
Phot table lenght: 78
No. of [Ne] data 2
[NeV] flux density (Jy):  [2010000000.0]
[NeII] flux density (Jy):  [3230000000.0]
[NeV]/[NeII]:  [0.6222910

No. of [Ne] data 7
[NeV] flux density (Jy):  [560000000.0]
[NeII] flux density (Jy):  [27500000000.0]
[NeV]/[NeII]:  [0.020363636363636365]
[NeV]/[NeII] errors:  [0.0010498474402684719]
Bibcodes:  ['2010ApJ...716.1151W']
Average [NeV]/[NeII]:  0.020363636363636365
Error:  0.0010498474402684719


143
Galaxy:  Mrk 1506
Phot table lenght: 467
No. of [Ne] data 11
[NeV] flux density (Jy):  [18300000000.0, 16600000000.0, 17200000000.0, 16400000000.0]
[NeII] flux density (Jy):  [8200000000.0, 7840000000.0, 9190000000.0, 7890000000.0]
[NeV]/[NeII]:  [2.231707317073171, 2.11734693877551, 1.8715995647442873, 2.0785804816223066]
[NeV]/[NeII] errors:  [nan, 0.2045381307726528, 0.13782291026225332, 0.10380434201948058]
Bibcodes:  ['2009ApJ...690.1105K', '2010ApJ...709.1257T', '2010ApJ...716.1151W', '2011ApJ...740...94D']
Average [NeV]/[NeII]:  2.074808575553819
Error:  0.06689840382775644


144
Galaxy:  NGC  6860
Phot table lenght: 140
No. of [Ne] data 7
[NeV] flux density (Jy):  [2850000000.0, 295

/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_257178/2360053561.py:42: RuntimeWarning: invalid value encountered in double_scalars


No. of [Ne] data 2
[NeV] flux density (Jy):  [4340000000.0]
[NeII] flux density (Jy):  [1390000000.0]
[NeV]/[NeII]:  [3.1223021582733814]
[NeV]/[NeII] errors:  [nan]
Bibcodes:  ['2006ApJ...649...79S']
Average [NeV]/[NeII]:  3.1223021582733814
Error:  nan


155
Galaxy:  2MASX J14510879+2709272
Phot table lenght: 99
No. of [Ne] data 2
[NeV] flux density (Jy):  [2670000000.0]
[NeII] flux density (Jy):  [507000000.0]
[NeV]/[NeII]:  [5.266272189349112]
[NeV]/[NeII] errors:  [nan]
Bibcodes:  ['2006ApJ...649...79S']
Average [NeV]/[NeII]:  5.266272189349112
Error:  nan


156
Galaxy:  MCG+13-07-002
Phot table lenght: 100
No. of [Ne] data 2
[NeV] flux density (Jy):  [324000000.0]
[NeII] flux density (Jy):  [411000000.0]
[NeV]/[NeII]:  [0.7883211678832117]
[NeV]/[NeII] errors:  [nan]
Bibcodes:  ['2006ApJ...649...79S']
Average [NeV]/[NeII]:  0.7883211678832117
Error:  nan


157
Galaxy:  LEDA    1790
Phot table lenght: 161
No. of [Ne] data 4
[NeV] flux density (Jy):  [473000000.0]
[NeII] flux densi

In [9]:
#Remove the galaxies where there was not info with same bibcode
NeV_NeII.remove_rows(np.where(np.isnan(NeV_NeII['[NeV]/[NeII]']))[0])
NeV_NeII_bib.remove_rows(np.where(np.isnan(NeV_NeII['[NeV]/[NeII]']))[0])

In [10]:
print('Final number of galaxies with [Ne V] and [Ne II] info: ', len(NeV_NeII))

Final number of galaxies with [Ne V] and [Ne II] info:  158


## [NeV]/[NeIII]

In [11]:
Nefilter = []
Refcode = []
NeV_NeIII = Table(names=('Main_id','RA','DEC','otype','redshift','[NeV]/[NeIII]','[NeV]/[NeIII]_err'),
                 dtype=('O','f8','f8','O','f8','f8','f8'))
NeV_NeIII_bib = Table(names=('Main_id','RA','DEC','otype','redshift','[NeV]/[NeIII]',\
                             '[NeV]/[NeIII]_err','Bibcode'),
                dtype=('O','f8','f8','O','f8','f8','f8','O'))

not_same_bib = 0

for id in range(len(NeV_NeIII_Sy)):
    NED_phot = Table()
    obj = NeV_NeIII_Sy['Main_Id'][id]
    ra = NeV_NeIII_Sy['RA'][id]
    dec = NeV_NeIII_Sy['DEC'][id]
    otype = NeV_NeIII_Sy['otype_txt'][id]
    z = NeV_NeIII_Sy['redshift'][id]
    print(id)
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    print('Phot table lenght:',len(NED_phot))
    
    #Put first the frequencies of the Ne line 1 and then the Ne line 2
    NeV_NeIII_Freq = [20900000000000.0,21000000000000.0,19200000000000.0,19300000000000.0]
    NeV_NeIII_inf = Neon_info(NED_phot, NeV_NeIII_Freq)
    
    NeV_NeIII_ratios,NeV_NeIII_ratios_err,NeV,NeIII,Ne_filter,refcode,not_same_bib = \
    Ne_ratios(NeV_NeIII_inf,'[Ne V]','[Ne III]','nan','[NeIII]',not_same_bib)
    
    print('[NeV] flux density (Jy): ',NeV)
    print('[NeIII] flux density (Jy): ',NeIII)
    print('[NeV]/[NeIII]: ',NeV_NeIII_ratios)
    print('[NeV]/[NeIII] errors: ',NeV_NeIII_ratios_err)
    print('Bibcodes: ',refcode)
    
    for r in range(len(refcode)): Refcode.append(refcode[r])
    for f in range(len(Ne_filter)): Nefilter.append(Ne_filter[f])
        
    avg_Ne_ratios = np.average(NeV_NeIII_ratios)
    std_Ne_ratios = np.std(NeV_NeIII_ratios)
    if len(NeV_NeIII_ratios)>1:
        NeV_NeIII_ratios_err = np.nan_to_num(NeV_NeIII_ratios_err)
    err = np.sqrt(np.sum(np.array(NeV_NeIII_ratios_err)**2))/len(NeV_NeIII_ratios_err)
    if err == 0: err = np.nan
    NeV_NeIII_row = [obj,ra,dec,otype,z,avg_Ne_ratios,err]
    NeV_NeIII.add_row(vals=NeV_NeIII_row)
    
    for b,ratio in enumerate(NeV_NeIII_ratios):
        NeV_NeIII_bib_row = [obj,ra,dec,otype,z,ratio,NeV_NeIII_ratios_err[b],refcode[b]]
        NeV_NeIII_bib.add_row(vals=NeV_NeIII_bib_row)
        
    print('Average [NeV]/[NeIII]: ',avg_Ne_ratios)
    print('Error: ',err)
    print('\n')

print('Number of galaxies with diferent bibcode for [NeV] and [NeIII] fluxes:',not_same_bib)
print('Galaxies idx with diferent bibcode for [NeV] and [NeIII] fluxes:',\
      np.where(np.isnan(NeV_NeIII['[NeV]/[NeIII]']))[0])

#Uncomment the 'for' lines for print the 'Observed_passband'and the Ŕefcodes' with [NeV] and [NeIII] information
Ne_filters2, countspb2 = np.unique(Nefilter, return_counts=True)
#for pb in range(len(Ne_filters2)): print(Ne_filters2[pb])
    
refcodes2, countsrc2 = np.unique(Refcode, return_counts=True)
#for rc in range(len(refcodes2)): print(refcodes2[rc])

0
Galaxy:  3C 234.0
Phot table lenght: 253
No. of [Ne] data 4
[NeV] flux density (Jy):  [3300000000.0, 2350000000.0]
[NeIII] flux density (Jy):  [8200000000.0, 3400000000.0]
[NeV]/[NeIII]:  [0.4024390243902439, 0.6911764705882353]
[NeV]/[NeIII] errors:  [0.09201936826462354, 0.100914932142928]
Bibcodes:  ['2006ApJ...647..161O', '2010ApJ...709.1257T']
Average [NeV]/[NeIII]:  0.5468077474892397
Error:  0.06828504167318082


1
Galaxy:  NGC   262
Phot table lenght: 334
No. of [Ne] data 7
[NeV] flux density (Jy):  [5820000000.0, 6900000000.0, 5820000000.0]
[NeIII] flux density (Jy):  [20400000000.0, 20600000000.0, 20400000000.0]
[NeV]/[NeIII]:  [0.2852941176470588, 0.33495145631067963, 0.2852941176470588]
[NeV]/[NeIII] errors:  [0.017961061968501214, 0.021688727467788264, nan]
Bibcodes:  ['2010ApJ...709.1257T', '2010ApJ...716.1151W', '2011ApJS..195...17W']
Average [NeV]/[NeIII]:  0.30184656386826575
Error:  0.009386755718140165


2
Galaxy:  IRAS 04385-0828
Phot table lenght: 97
No. of [Ne] 

No. of [Ne] data 6
[NeV] flux density (Jy):  [1440000000.0, 847000000.0]
[NeIII] flux density (Jy):  [2630000000.0, 2560000000.0]
[NeV]/[NeIII]:  [0.5475285171102662, 0.330859375]
[NeV]/[NeIII] errors:  [nan, 0.04484808550523028]
Bibcodes:  ['2009ApJ...690.1105K', '2011ApJ...740...94D']
Average [NeV]/[NeIII]:  0.43919394605513307
Error:  0.02242404275261514


22
Galaxy:  Mrk    9
Phot table lenght: 137
No. of [Ne] data 3
[NeV] flux density (Jy):  [2210000000.0]
[NeIII] flux density (Jy):  [1900000000.0]
[NeV]/[NeIII]:  [1.1631578947368422]
[NeV]/[NeIII] errors:  [nan]
Bibcodes:  ['2011ApJS..195...17W']
Average [NeV]/[NeIII]:  1.1631578947368422
Error:  nan


23
Galaxy:  Mrk 1513
Phot table lenght: 216
No. of [Ne] data 6
[NeV] flux density (Jy):  [3800000000.0, 4520000000.0]
[NeIII] flux density (Jy):  [6190000000.0, 5570000000.0]
[NeV]/[NeIII]:  [0.6138933764135702, 0.8114901256732495]
[NeV]/[NeIII] errors:  [nan, 0.06841628227154098]
Bibcodes:  ['2009ApJ...690.1105K', '2011ApJ...740..

/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_257178/307791065.py:44: RuntimeWarning: invalid value encountered in double_scalars
 

Phot table lenght: 271
No. of [Ne] data 11
[NeV] flux density (Jy):  [7300000000.0, 5080000000.0, 5010000000.0, 6050000000.0, 5010000000.0]
[NeIII] flux density (Jy):  [8600000000.0, 6760000000.0, 5880000000.0, 6620000000.0, 5880000000.0]
[NeV]/[NeIII]:  [0.8488372093023255, 0.7514792899408284, 0.8520408163265306, 0.9138972809667674, 0.8520408163265306]
[NeV]/[NeIII] errors:  [nan, nan, 0.11920855509748761, 0.1235123298283933, nan]
Bibcodes:  ['2007ApJ...655L..73G', '2009ApJ...690.1105K', '2010ApJ...709.1257T', '2010ApJ...716.1151W', '2011ApJS..195...17W']
Average [NeV]/[NeIII]:  0.8436590825725965
Error:  0.034331312371110166


35
Galaxy:  NGC   973
Phot table lenght: 105
No. of [Ne] data 2
[NeV] flux density (Jy):  [2070000000.0]
[NeIII] flux density (Jy):  [8920000000.0]
[NeV]/[NeIII]:  [0.2320627802690583]
[NeV]/[NeIII] errors:  [0.02247090765432689]
Bibcodes:  ['2010ApJ...716.1151W']
Average [NeV]/[NeIII]:  0.2320627802690583
Error:  0.02247090765432689


36
Galaxy:  2E  3648
Phot

Phot table lenght: 140
No. of [Ne] data 2
[NeV] flux density (Jy):  []
[NeIII] flux density (Jy):  []
[NeV]/[NeIII]:  []
[NeV]/[NeIII] errors:  []
Bibcodes:  []
Average [NeV]/[NeIII]:  nan
Error:  nan


57
Galaxy:  Mrk 1095
Phot table lenght: 202
No. of [Ne] data 3
[NeV] flux density (Jy):  [1000000000.0]
[NeIII] flux density (Jy):  [3520000000.0]
[NeV]/[NeIII]:  [0.2840909090909091]
[NeV]/[NeIII] errors:  [0.050699085331481264]
Bibcodes:  ['2011ApJ...740...94D']
Average [NeV]/[NeIII]:  0.2840909090909091
Error:  0.050699085331481264


58
Galaxy:  IC  486
Phot table lenght: 130
No. of [Ne] data 2
[NeV] flux density (Jy):  [3300000000.0]
[NeIII] flux density (Jy):  [6790000000.0]
[NeV]/[NeIII]:  [0.48600883652430044]
[NeV]/[NeIII] errors:  [0.033068698253822096]
Bibcodes:  ['2010ApJ...716.1151W']


...

No. of [Ne] data 4
[NeV] flux density (Jy):  [710000000.0, 1680000000.0]
[NeIII] flux density (Jy):  [4730000000.0, 2990000000.0]
[NeV]/[NeIII]:  [0.15010570824524314, 0.5618729096989966]
[NeV]/[NeIII] errors:  [nan, 0.0834931740022381]
Bibcodes:  ['2011ApJS..193...18W', '2012ApJ...747...95G']
Average [NeV]/[NeIII]:  0.3559893089721199
Error:  0.04174658700111905


96
Galaxy:  ESO 362-18
Phot table lenght: 190
No. of [Ne] data 6
[NeV] flux density (Jy):  [3270000000.0, 3150000000.0]
[NeIII] flux density (Jy):  [7490000000.0, 7160000000.0]
[NeV]/[NeIII]:  [0.43658210947930576, 0.4399441340782123]
[NeV]/[NeIII] errors:  [0.02890784380195692, 0.05284629911000138]
Bibcodes:  ['2010ApJ...709.1257T', '2010ApJ...716.1151W']
Average [NeV]/[NeIII]:  0.43826312177875903
Error:  0.030118079134060293


97
Galaxy:  IC  450
Phot table lenght: 233
No. of [Ne] data 4
[NeV] flux density (Jy):  [12000000000.0, 9390000000.0]
[NeIII] flux density (Jy):  [47000000000.0, 49300000000.0]
[NeV]/[NeIII]:  [0.25

No. of [Ne] data 2
[NeV] flux density (Jy):  [6000000000.0]
[NeIII] flux density (Jy):  [8000000000.0]
[NeV]/[NeIII]:  [0.75]
[NeV]/[NeIII] errors:  [0.15625]
Bibcodes:  ['2007ApJ...671..124D']
Average [NeV]/[NeIII]:  0.75
Error:  0.15625


114
Galaxy:  IC 2560
Phot table lenght: 122
No. of [Ne] data 2
[NeV] flux density (Jy):  [19000000000.0]
[NeIII] flux density (Jy):  [36000000000.0]
[NeV]/[NeIII]:  [0.5277777777777778]
[NeV]/[NeIII] errors:  [nan]
Bibcodes:  ['2010ApJ...725.2270P']
Average [NeV]/[NeIII]:  0.5277777777777778
Error:  nan


115
Galaxy:  NGC  5995
Phot table lenght: 153
No. of [Ne] data 8
[NeV] flux density (Jy):  [12000000000.0, 6130000000.0, 7600000000.0, 6130000000.0]
[NeIII] flux density (Jy):  [18000000000.0, 8470000000.0, 9190000000.0, 8470000000.0]
[NeV]/[NeIII]:  [0.6666666666666666, 0.7237308146399055, 0.8269858541893362, 0.7237308146399055]
[NeV]/[NeIII] errors:  [0.09259259259259259, 0.05348623498677602, 0.1633828554572338, nan]
Bibcodes:  ['2007ApJ...671..1

/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_257178/307791065.py:44: RuntimeWarning: invalid value encountered in double_scalars
 

No. of [Ne] data 10
[NeV] flux density (Jy):  [109000000000.0, 64600000000.0, 63300000000.0, 109000000000.0]
[NeIII] flux density (Jy):  [207000000000.0, 182000000000.0, 175000000000.0, 207000000000.0]
[NeV]/[NeIII]:  [0.5265700483091788, 0.35494505494505496, 0.3617142857142857, 0.5265700483091788]
[NeV]/[NeIII] errors:  [0.09382298335608735, nan, 0.004851469095010472, nan]
Bibcodes:  ['2007ApJ...671..124D', '2009ApJ...690.1105K', '2011ApJ...740...94D', '2011ApJS..195...17W']
Average [NeV]/[NeIII]:  0.4424498593194245
Error:  0.02348708283053752


120
Galaxy:  NGC  4051
Phot table lenght: 425
No. of [Ne] data 15
[NeV] flux density (Jy):  [18000000000.0, 11300000000.0, 10700000000.0, 10200000000.0, 11000000000.0, 10800000000.0, 10700000000.0]
[NeIII] flux density (Jy):  [23000000000.0, 16000000000.0, 17100000000.0, 16400000000.0, 16000000000.0, 16200000000.0, 17100000000.0]
[NeV]/[NeIII]:  [0.782608695652174, 0.70625, 0.6257309941520468, 0.6219512195121951, 0.6875, 0.6666666666666666, 0

No. of [Ne] data 6
[NeV] flux density (Jy):  [2540000000.0, 1890000000.0, 2540000000.0]
[NeIII] flux density (Jy):  [4820000000.0, 4720000000.0, 4820000000.0]
[NeV]/[NeIII]:  [0.5269709543568465, 0.4004237288135593, 0.5269709543568465]
[NeV]/[NeIII] errors:  [0.07605154066481172, 0.029994080070188396, nan]
Bibcodes:  ['2010ApJ...709.1257T', '2010ApJ...716.1151W', '2011ApJS..195...17W']
Average [NeV]/[NeIII]:  0.4847885458424175
Error:  0.027250854577320415


142
Galaxy:  NGC  7213
Phot table lenght: 263
No. of [Ne] data 6
[NeV] flux density (Jy):  [560000000.0]
[NeIII] flux density (Jy):  [12800000000.0]
[NeV]/[NeIII]:  [0.04375]
[NeV]/[NeIII] errors:  [0.0023873108474338632]
Bibcodes:  ['2010ApJ...716.1151W']
Average [NeV]/[NeIII]:  0.04375
Error:  0.0023873108474338632


143
Galaxy:  Mrk 1506
Phot table lenght: 467
No. of [Ne] data 11
[NeV] flux density (Jy):  [21700000000.0, 18300000000.0, 16600000000.0, 17200000000.0, 16400000000.0]
[NeIII] flux density (Jy):  [35300000000.0, 29900

In [12]:
#Remove the galaxies where there was not info with same bibcode
NeV_NeIII.remove_rows(np.where(np.isnan(NeV_NeIII['[NeV]/[NeIII]']))[0])
NeV_NeIII_bib.remove_rows(np.where(np.isnan(NeV_NeIII['[NeV]/[NeIII]']))[0])

print('Final number of galaxies with [Ne V] and [Ne III] info: ', len(NeV_NeIII))

Final number of galaxies with [Ne V] and [Ne III] info:  153


## [NeIII]/[NeII]

In [13]:
Nefilter = []
Refcode = []
NeIII_NeII = Table(names=('Main_id','RA','DEC','otype','redshift','[NeIII]/[NeII]','[NeIII]/[NeII]_err'),
                   dtype=('O','f8','f8','O','f8','f8','f8'))
NeIII_NeII_bib = Table(names=('Main_id','RA','DEC','otype','redshift','[NeIII]/[NeII]','[NeIII]/[NeII]_err','Bibcode'),
                dtype=('O','f8','f8','O','f8','f8','f8','O'))

not_same_bib = 0

for id in range(len(NeIII_NeII_Sy)):
    NED_phot = Table()
    obj = NeIII_NeII_Sy['Main_Id'][id]
    ra = NeIII_NeII_Sy['RA'][id]
    dec = NeIII_NeII_Sy['DEC'][id]
    otype = NeIII_NeII_Sy['otype_txt'][id]
    z = NeIII_NeII_Sy['redshift'][id]
    print(id)
    print('Galaxy: ',obj)

    NED_phot = Table.read(path_phot_ned+obj+'_NED_phot_tables'+'.vot', format='votable')
    print('Phot table lenght:',len(NED_phot))
    
    #Put first the frequencies of the Ne line 1 and then the Ne line 2
    NeIII_NeII_Freq = [19200000000000.0,19300000000000.0,23400000000000.0]
    NeIII_NeII_inf = Neon_info(NED_phot, NeIII_NeII_Freq)
    
    NeIII_NeII_ratios,NeIII_NeII_ratios_err,NeIII,NeII,Ne_filter,refcode,not_same_bib = \
    Ne_ratios(NeIII_NeII_inf,'[Ne III]','[Ne II]','[NeIII]','nan',not_same_bib)
    
    print('[NeIII] flux density (Jy): ',NeIII)
    print('[NeII] flux density (Jy): ',NeII)
    print('[NeIII]/[NeII]: ',NeIII_NeII_ratios)
    print('[NeIII]/[NeII] errors: ',NeIII_NeII_ratios_err)
    print('Bibcodes: ',refcode)
    
    for r in range(len(refcode)): Refcode.append(refcode[r])
    for f in range(len(Ne_filter)): Nefilter.append(Ne_filter[f])
    
    avg_Ne_ratios = np.average(NeIII_NeII_ratios)
    std_Ne_ratios = np.std(NeIII_NeII_ratios)
    if len(NeIII_NeII_ratios)>1:
        NeIII_NeII_ratios_err = np.nan_to_num(NeIII_NeII_ratios_err)
    err = np.sqrt(np.sum(np.array(NeIII_NeII_ratios_err)**2))/len(NeIII_NeII_ratios_err)
    if err == 0: err = np.nan
    NeIII_NeII_row = [obj,ra,dec,otype,z,avg_Ne_ratios,err]
    NeIII_NeII.add_row(vals=NeIII_NeII_row)
    
    for b,ratio in enumerate(NeIII_NeII_ratios):
        NeIII_NeII_bib_row = [obj,ra,dec,otype,z,ratio,NeIII_NeII_ratios_err[b],refcode[b]]
        NeIII_NeII_bib.add_row(vals=NeIII_NeII_bib_row)
    
    print('Average [NeIII]/[NeII]: ',avg_Ne_ratios)
    print('Error: ',err)
    print('\n')

print('Number of galaxies with diferent bibcode for [NeIII] and [NeII] fluxes:',not_same_bib)
print('Galaxies idx with diferent bibcode for [NeIII] and [NeII] fluxes:',\
      np.where(np.isnan(NeIII_NeII['[NeIII]/[NeII]']))[0])

#Uncomment the 'for' lines for print the 'Observed_passband'and the Ŕefcodes' with [NeIII] and [NeII] information
Ne_filters3, countspb3 = np.unique(Nefilter, return_counts=True)
#for pb in range(len(Ne_filters3)): print(Ne_filters3[pb])

refcodes3, countsrc3 = np.unique(Refcode, return_counts=True)
#for rc in range(len(refcodes3)): print(refcodes3[rc])

0
Galaxy:  3C 234.0
Phot table lenght: 253
No. of [Ne] data 4
[NeIII] flux density (Jy):  [8200000000.0]
[NeII] flux density (Jy):  [800000000.0]
[NeIII]/[NeII]:  [10.25]
[NeIII]/[NeII] errors:  [2.7077723778043086]
Bibcodes:  ['2006ApJ...647..161O']
Average [NeIII]/[NeII]:  10.25
Error:  2.7077723778043086


1
Galaxy:  NGC   262
Phot table lenght: 334
No. of [Ne] data 8
[NeIII] flux density (Jy):  [25000000000.0, 20400000000.0, 20600000000.0, 20400000000.0]
[NeII] flux density (Jy):  [13000000000.0, 16400000000.0, 15300000000.0, 16400000000.0]
[NeIII]/[NeII]:  [1.9230769230769231, 1.2439024390243902, 1.34640522875817, 1.2439024390243902]
[NeIII]/[NeII] errors:  [0.16673376098941267, 0.03523648682068849, 0.08310665402417797, nan]
Bibcodes:  ['2007ApJ...671..124D', '2010ApJ...709.1257T', '2010ApJ...716.1151W', '2011ApJS..195...17W']
Average [NeIII]/[NeII]:  1.4393217574709685
Error:  0.04740020635519003


2
Galaxy:  IRAS 04385-0828
Phot table lenght: 97
No. of [Ne] data 6
[NeIII] flux d

No. of [Ne] data 2
[NeIII] flux density (Jy):  [20400000000.0]
[NeII] flux density (Jy):  [27300000000.0]
[NeIII]/[NeII]:  [0.7472527472527473]
[NeIII]/[NeII] errors:  [nan]
Bibcodes:  ['2011ApJS..195...17W']
Average [NeIII]/[NeII]:  0.7472527472527473
Error:  nan


21
Galaxy:  Mrk 1383
Phot table lenght: 205
No. of [Ne] data 5
[NeIII] flux density (Jy):  [2630000000.0, 2560000000.0]
[NeII] flux density (Jy):  [1230000000.0, 1350000000.0]
[NeIII]/[NeII]:  [2.138211382113821, 1.8962962962962964]
[NeIII]/[NeII] errors:  [nan, 0.1718519157088004]
Bibcodes:  ['2009ApJ...690.1105K', '2011ApJ...740...94D']
Average [NeIII]/[NeII]:  2.0172538392050585
Error:  0.0859259578544002


22
Galaxy:  Mrk    9
Phot table lenght: 137
No. of [Ne] data 5
[NeIII] flux density (Jy):  [1000000000.0, 1900000000.0]
[NeII] flux density (Jy):  [2000000000.0, 3230000000.0]
[NeIII]/[NeII]:  [0.5, 0.5882352941176471]
[NeIII]/[NeII] errors:  [0.5590169943749475, nan]
Bibcodes:  ['2007ApJ...671..124D', '2011ApJS..195.

No. of [Ne] data 2
[NeIII] flux density (Jy):  [16300000000.0]
[NeII] flux density (Jy):  [11500000000.0]
[NeIII]/[NeII]:  [1.4173913043478261]
[NeIII]/[NeII] errors:  [nan]
Bibcodes:  ['2011ApJS..195...17W']
Average [NeIII]/[NeII]:  1.4173913043478261
Error:  nan


41
Galaxy:  UGC 12282
Phot table lenght: 52
No. of [Ne] data 2
[NeIII] flux density (Jy):  [1760000000.0]
[NeII] flux density (Jy):  [2080000000.0]
[NeIII]/[NeII]:  [0.8461538461538461]
[NeIII]/[NeII] errors:  [0.11641422077045022]
Bibcodes:  ['2010ApJ...716.1151W']
Average [NeIII]/[NeII]:  0.8461538461538461
Error:  0.11641422077045022


42
Galaxy:  MCG+10-17-021
Phot table lenght: 94
No. of [Ne] data 2
[NeIII] flux density (Jy):  [10500000000.0]
[NeII] flux density (Jy):  [10800000000.0]
[NeIII]/[NeII]:  [0.9722222222222222]
[NeIII]/[NeII] errors:  [0.04381231303644306]
Bibcodes:  ['2011ApJ...740...94D']
Average [NeIII]/[NeII]:  0.9722222222222222
Error:  0.04381231303644306


43
Galaxy:  ESO 113-45
Phot table lenght: 199

No. of [Ne] data 6
[NeIII] flux density (Jy):  [9000000000.0, 11100000000.0, 5670000000.0]
[NeII] flux density (Jy):  [12000000000.0, 22200000000.0, 21400000000.0]
[NeIII]/[NeII]:  [0.75, 0.5, 0.2649532710280374]
[NeIII]/[NeII] errors:  [0.20833333333333334, 0.01924325167864309, 0.01409589106860697]
Bibcodes:  ['2007ApJ...671..124D', '2010ApJ...725..605S', '2011ApJ...740...94D']
Average [NeIII]/[NeII]:  0.5049844236760125
Error:  0.06989815977387183


61
Galaxy:  NGC  3079
Phot table lenght: 331
No. of [Ne] data 11
[NeIII] flux density (Jy):  [28000000000.0, 23200000000.0, 24000000000.0, 23800000000.0, 23200000000.0]
[NeII] flux density (Jy):  [148000000000.0, 99000000000.0, 108000000000.0, 131000000000.0, 99000000000.0]
[NeIII]/[NeII]:  [0.1891891891891892, 0.23434343434343435, 0.2222222222222222, 0.18167938931297709, 0.23434343434343435]
[NeIII]/[NeII] errors:  [0.013342150809063428, 0.006684295848125313, nan, 0.00869413846830901, nan]
Bibcodes:  ['2007ApJ...671..124D', '2009ApJS..18

No. of [Ne] data 4
[NeIII] flux density (Jy):  [142000000000.0]
[NeII] flux density (Jy):  [50800000000.0]
[NeIII]/[NeII]:  [2.795275590551181]
[NeIII]/[NeII] errors:  [nan]
Bibcodes:  ['2009ApJ...690.1105K']
Average [NeIII]/[NeII]:  2.795275590551181
Error:  nan


79
Galaxy:  NGC  5347
Phot table lenght: 154
No. of [Ne] data 7
[NeIII] flux density (Jy):  [4430000000.0, 4090000000.0, 4090000000.0]
[NeII] flux density (Jy):  [5190000000.0, 4170000000.0, 4170000000.0]
[NeIII]/[NeII]:  [0.8535645472061657, 0.9808153477218226, 0.9808153477218226]
[NeIII]/[NeII] errors:  [nan, 0.26208825604524505, nan]
Bibcodes:  ['2009ApJ...690.1105K', '2010ApJ...709.1257T', '2011ApJS..195...17W']
Average [NeIII]/[NeII]:  0.9383984142166036
Error:  0.08736275201508169


80
Galaxy:  NGC  4748
Phot table lenght: 158
No. of [Ne] data 7
[NeIII] flux density (Jy):  [40000000000.0, 37800000000.0, 15900000000.0]
[NeII] flux density (Jy):  [14000000000.0, 11400000000.0, 7370000000.0]
[NeIII]/[NeII]:  [2.8571428571

...

/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_257178/1785080582.py:43: RuntimeWarning: invalid value encountered in double_scalars


Phot table lenght: 414
No. of [Ne] data 6
[NeIII] flux density (Jy):  [70400000000.0, 60000000000.0]
[NeII] flux density (Jy):  [193000000000.0, 187000000000.0]
[NeIII]/[NeII]:  [0.36476683937823834, 0.32085561497326204]
[NeIII]/[NeII] errors:  [0.014266612257987544, 0.00545924988171952]
Bibcodes:  ['2006ApJ...640..204A', '2011ApJ...740...94D']
Average [NeIII]/[NeII]:  0.3428112271757502
Error:  0.007637729286096998


112
Galaxy:  NGC  7603
Phot table lenght: 187
No. of [Ne] data 7
[NeIII] flux density (Jy):  [7000000000.0, 5070000000.0, 7000000000.0]
[NeII] flux density (Jy):  [11000000000.0, 9320000000.0, 11000000000.0]
[NeIII]/[NeII]:  [0.6363636363636364, 0.5439914163090128, 0.6363636363636364]
[NeIII]/[NeII] errors:  [0.10775541165624211, 0.06730713130748388, nan]
Bibcodes:  ['2007ApJ...671..124D', '2010ApJ...716.1151W', '2011ApJS..195...17W']
Average [NeIII]/[NeII]:  0.6055728963454285
Error:  0.0423497063692419


113
Galaxy:  UGC 12138
Phot table lenght: 120
No. of [Ne] data 2
[

Phot table lenght: 404
No. of [Ne] data 4
[NeIII] flux density (Jy):  [5080000000.0, 14000000000.0]
[NeII] flux density (Jy):  [13300000000.0, 31000000000.0]
[NeIII]/[NeII]:  [0.3819548872180451, 0.45161290322580644]
[NeIII]/[NeII] errors:  [0.012406429922928192, nan]
Bibcodes:  ['2009MNRAS.398.1165G', '2010ApJ...725.2270P']
Average [NeIII]/[NeII]:  0.41678389522192577
Error:  0.006203214961464096


134
Galaxy:  MCG-03-58-007
Phot table lenght: 92
No. of [Ne] data 6
[NeIII] flux density (Jy):  [10000000000.0, 9290000000.0, 9290000000.0]
[NeII] flux density (Jy):  [5000000000.0, 8520000000.0, 8520000000.0]
[NeIII]/[NeII]:  [2.0, 1.09037558685446, 1.09037558685446]
[NeIII]/[NeII] errors:  [1.61245154965971, 0.06147580189039626, nan]
Bibcodes:  ['2007ApJ...671..124D', '2010ApJ...709.1257T', '2011ApJS..195...17W']
Average [NeIII]/[NeII]:  1.39358372456964
Error:  0.53787434243181


135
Galaxy:  NGC  4922
Phot table lenght: 87
No. of [Ne] data 4
[NeIII] flux density (Jy):  [8860000000.0, 92

No. of [Ne] data 4
[NeIII] flux density (Jy):  [8690000000.0, 6890000000.0]
[NeII] flux density (Jy):  [67800000000.0, 61100000000.0]
[NeIII]/[NeII]:  [0.12817109144542774, 0.1127659574468085]
[NeIII]/[NeII] errors:  [0.00887486743132394, nan]
Bibcodes:  ['2007ApJ...656..148A', '2011ApJS..193...18W']
Average [NeIII]/[NeII]:  0.12046852444611812
Error:  0.00443743371566197


152
Galaxy:  LEDA   89032
Phot table lenght: 69
No. of [Ne] data 2
[NeIII] flux density (Jy):  [1630000000.0]
[NeII] flux density (Jy):  [1450000000.0]
[NeIII]/[NeII]:  [1.1241379310344828]
[NeIII]/[NeII] errors:  [nan]
Bibcodes:  ['2009ApJ...690.1105K']
Average [NeIII]/[NeII]:  1.1241379310344828
Error:  nan


153
Galaxy:  NGC  7479
Phot table lenght: 207
No. of [Ne] data 2
[NeIII] flux density (Jy):  [5800000000.0]
[NeII] flux density (Jy):  [9400000000.0]
[NeIII]/[NeII]:  [0.6170212765957447]
[NeIII]/[NeII] errors:  [nan]
Bibcodes:  ['2010ApJ...725.2270P']
Average [NeIII]/[NeII]:  0.6170212765957447
Error:  nan



No. of [Ne] data 2
[NeIII] flux density (Jy):  [1100000000.0]
[NeII] flux density (Jy):  [2300000000.0]
[NeIII]/[NeII]:  [0.4782608695652174]
[NeIII]/[NeII] errors:  [nan]
Bibcodes:  ['2010ApJ...725.2270P']
Average [NeIII]/[NeII]:  0.4782608695652174
Error:  nan


175
Galaxy:  M  58
Phot table lenght: 433
No. of [Ne] data 3
[NeIII] flux density (Jy):  [12000000000.0]
[NeII] flux density (Jy):  [23000000000.0]
[NeIII]/[NeII]:  [0.5217391304347826]
[NeIII]/[NeII] errors:  [nan]
Bibcodes:  ['2010ApJ...725.2270P']
Average [NeIII]/[NeII]:  0.5217391304347826
Error:  nan


176
Galaxy:  2MASX J14254352+3345275
Phot table lenght: 100
No. of [Ne] data 2
[NeIII] flux density (Jy):  [860000000.0]
[NeII] flux density (Jy):  [3400000000.0]
[NeIII]/[NeII]:  [0.2529411764705882]
[NeIII]/[NeII] errors:  [0.035771284224731234]
Bibcodes:  ['2007ApJ...671..323H']
Average [NeIII]/[NeII]:  0.2529411764705882
Error:  0.035771284224731234


177
Galaxy:  NGC  7743
Phot table lenght: 148
No. of [Ne] data 2
[Ne

In [14]:
#Remove the galaxies where there was not info with same bibcode
NeIII_NeII.remove_rows(np.where(np.isnan(NeIII_NeII['[NeIII]/[NeII]']))[0])
NeIII_NeII_bib.remove_rows(np.where(np.isnan(NeIII_NeII['[NeIII]/[NeII]']))[0])

print('Final number of galaxies with [Ne III] and [Ne II] info: ', len(NeIII_NeII))

Final number of galaxies with [Ne III] and [Ne II] info:  190


----------------------------------
## Joining tables with CIGALE results:

If we want to join to the different CIGALE results after the data cleaning from Notebook 5, we must to change the file name in the next cell:

- results0a90_All.fits
- results30y70_All.fits
- results_bestChi.fits
- resultsDBIC_All.fits

In [15]:
CIGresults = Table.read(path_CIGALE_out+'results0a90_All.fits', format='fits')
#CIGresults

In [16]:
CIGresults = CIGresults['id','bayes.agn.fracAGN','bayes.agn.fracAGN_err','bayes.agn.i',\
                        'bayes.agn.i_err','bayes.agn.disk_luminosity','bayes.agn.disk_luminosity_err',\
                        'bayes.sfh.sfr','bayes.sfh.sfr_err','bayes.stellar.m_star',\
                        'bayes.stellar.m_star_err','best.reduced_chi_square','log.reduced_chi_square']
CIGresults['id'] = CIGresults['id'].astype(str) #Converting the dtype to string
CIGresults['id'] = CIGresults['id'].astype(object) #Converting the dtype to object
CIGresults['id'].name = 'Main_id' #Changing the column name
len(CIGresults)

118

In [17]:
NeVNeII = join(NeV_NeII, CIGresults, join_type='left', keys='Main_id')
NeVNeII = NeVNeII.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeVNeII.remove_rows(np.where(np.isnan(NeVNeII['bayes.agn.i']))[0])
#NeVNeII.show_in_notebook()

In [18]:
NeVNeIII = join(NeV_NeIII, CIGresults, join_type='left', keys='Main_id')
NeVNeIII = NeVNeIII.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeVNeIII.remove_rows(np.where(np.isnan(NeVNeIII['bayes.agn.i']))[0])
#NeVNeIII.show_in_notebook()

In [19]:
NeIIINeII = join(NeIII_NeII, CIGresults, join_type='left', keys='Main_id')
NeIIINeII = NeIIINeII.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeIIINeII.remove_rows(np.where(np.isnan(NeIIINeII['bayes.agn.i']))[0])
#NeIIINeII.show_in_notebook()

In [20]:
NeVNeIINeIII = join(NeVNeII, NeVNeIII, join_type='outer')
Sy_Ne = join(NeVNeIINeIII, NeIIINeII, join_type='outer')
Sy_Ne = Sy_Ne.filled(np.nan) #Fill the mask values (---) with numpy.nan

Sy_Ne = Sy_Ne['Main_id','RA','DEC','otype','redshift','[NeV]/[NeII]','[NeV]/[NeII]_err',\
              '[NeV]/[NeIII]','[NeV]/[NeIII]_err','[NeIII]/[NeII]','[NeIII]/[NeII]_err',\
              'bayes.agn.fracAGN','bayes.agn.fracAGN_err','bayes.agn.i','bayes.agn.i_err',\
              'bayes.agn.disk_luminosity','bayes.agn.disk_luminosity_err',\
              'bayes.sfh.sfr','bayes.sfh.sfr_err','bayes.stellar.m_star','bayes.stellar.m_star_err',\
              'best.reduced_chi_square','log.reduced_chi_square'] #reorder columns

Sy_Ne.write(path_FINAL+'Ne_ratios_-_CIGALE_results.tbl', format='ascii',overwrite=True)
#Sy_Ne.write(path_FINAL+'Ne_ratios_-_CIGALE_results.fits', format='fits',overwrite=True)

-----------------------------
### By bibcode

In [21]:
NeVNeII_bib = join(NeV_NeII_bib, CIGresults, join_type='left', keys='Main_id')
NeVNeII_bib = NeVNeII_bib.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeVNeII_bib.remove_rows(np.where(np.isnan(NeVNeII_bib['bayes.agn.i']))[0])
NeVNeIII_bib = join(NeV_NeIII_bib, CIGresults, join_type='left', keys='Main_id')
NeVNeIII_bib = NeVNeIII_bib.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeVNeIII_bib.remove_rows(np.where(np.isnan(NeVNeIII_bib['bayes.agn.i']))[0])
NeIIINeII_bib = join(NeIII_NeII_bib, CIGresults, join_type='left', keys='Main_id')
NeIIINeII_bib = NeIIINeII_bib.filled(np.nan) #Fill the mask values (---) with numpy.nan
NeIIINeII_bib.remove_rows(np.where(np.isnan(NeIIINeII_bib['bayes.agn.i']))[0])
NeVNeIINeIII_bib = join(NeVNeII_bib, NeVNeIII_bib, join_type='outer')
Sy_Ne_bib = join(NeVNeIINeIII_bib, NeIIINeII_bib, join_type='outer')
Sy_Ne_bib = Sy_Ne_bib.filled(np.nan) #Fill the mask values (---) with numpy.nan

Sy_Ne_bib = Sy_Ne_bib['Main_id','RA','DEC','otype','redshift','[NeV]/[NeII]','[NeV]/[NeII]_err',\
              '[NeV]/[NeIII]','[NeV]/[NeIII]_err','[NeIII]/[NeII]','[NeIII]/[NeII]_err',\
              'bayes.agn.fracAGN','bayes.agn.fracAGN_err','bayes.agn.i','bayes.agn.i_err',\
              'bayes.sfh.sfr','bayes.sfh.sfr_err','bayes.stellar.m_star','bayes.stellar.m_star_err',\
              'best.reduced_chi_square','log.reduced_chi_square'] #reorder columns

Sy_Ne_bib.write(path_FINAL+'Ne_ratios_-_CIGALE_results_bib.tbl', format='ascii',overwrite=True)
#Sy_Ne_bib.show_in_notebook()

-------------------------------
## Notebook Info

In [22]:
%reload_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 7.31.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.15.0-58-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 5.1

matplotlib: 3.5.2
astropy   : 5.1
numpy     : 1.21.5

Watermark: 2.3.1

